### 1. LightFM 구현

In [19]:
# 필요한 라이브러리 불러오기
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
import time

#### 1-1. Dataset

In [20]:
# ✅ 1. Movielens 데이터셋 불러오기 (평점 4점 이상만 사용)
data = fetch_movielens(indicator_features=False, genre_features=True)
# LightFM이 사용하는 MovieLens 100k 원본 파일 경로
url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.item"
# 해당 파일에는 | 구분자, 인코딩은 ISO-8859-1 사용
movies = pd.read_csv(url, sep='|', header=None, encoding='latin-1')
# 영화 제목만 추출
item_labels = movies[1].values
# 원본 데이터 대입
data['item_labels'] = item_labels

#### 1-2. Model

In [ ]:
# ✅ 2. LightFM 모델 학습 (WARP Loss 은 활용하지 못해서 logistic 활용)
model = LightFM(loss='logistic')

In [22]:
try:
    start = time.time()
    model.fit(data['train'], epochs=1, num_threads=1)
    print("✅ 3단계: 모델 학습 완료!", f"({time.time() - start:.2f}초)")
except Exception as e:
    print("❌ model.fit 중 오류 발생:", e)

✅ 3단계: 모델 학습 완료! (0.15초)


#### 1-3. 함수 정의 및 출력

In [23]:
# ✅ 3. 추천 함수 정의
def recommend_movies(model, user_id, num_items=5):
    n_items = data['train'].shape[1]
    scores = model.predict(user_id, np.arange(n_items))
    top_items = np.argsort(-scores)[:num_items]  # 높은 점수순으로 정렬
    return [data['item_labels'][i] for i in top_items]

# ✅ 4. 특정 사용자에게 추천 실행
user_id = 3
recommended_movies = recommend_movies(model, user_id)

print(f"\n✅ 사용자 {user_id}에게 추천하는 영화:")
for i, movie in enumerate(recommended_movies, start=1):
    print(f"{i}. {movie}")


✅ 사용자 3에게 추천하는 영화:
1. Star Wars (1977)
2. Fargo (1996)
3. Return of the Jedi (1983)
4. Contact (1997)
5. English Patient, The (1996)


### 2. Hybrid 모델로 확장

#### 2-1. 유저 메타데이터 로드 및 전처리

In [24]:
# 유저 정보 로딩
user_info_url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.user"
users = pd.read_csv(user_info_url, sep='|', header=None, encoding='latin-1')
users.columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

# user_id를 0부터 시작하게 맞춤
users['user_id'] = users['user_id'] -1
users = users.sort_values('user_id').reset_index(drop=True)

# 직업과 성별을 범주형으로 인코딩 (예: "gender=F", "occupation=student")
users['gender'] ='gender=' + users['gender']
users['occupation'] ='occupation=' + users['occupation']

# 특성 컬럼만 선택
user_features_raw = users[['gender','occupation']]

#### 2-2. LightFM용 user_features 만들기(CSR 포맷)

In [ ]:
from lightfm.data import Dataset

# 새 데이터셋 생성
dataset = Dataset()

# 사용자와 아이템 수 정의
num_users, num_items = data['train'].shape
dataset.fit(users=range(num_users), items=range(num_items))

# 유저 특성 등록
all_user_features = list(np.unique(user_features_raw.values.flatten()))
dataset.fit_partial(user_features=all_user_features)

# 사용자별 특성 맵핑
user_features_tuple = [
    (user_id, list(user_features_raw.iloc[user_id].values)) for user_id in range(num_users)
]

# CSR 행렬 생성
user_features = dataset.build_user_features(user_features_tuple)

#### 2-3. 모델에 유저 특성 추가해서 학습

In [34]:
# 모델 새로 정의
model_hybrid = LightFM(loss ='logistic')

# hybrid 모델 학습
model_hybrid.fit(data['train'],
                 user_features=user_features,
                 item_features=data['item_features'],
                 epochs=4,
                 num_threads=2)

#### 2-4. 추천 함수 수정 (user_features, item_features 포함)

In [39]:
def recommend_hybrid(model, user_id, num_items=5):
    n_items = data['train'].shape[1]
    scores = model.predict(user_id, np.arange(n_items),
                           user_features=user_features,
                           item_features=data['item_features'])
    top_items = np.argsort(-scores)[:num_items]
    return [data['item_labels'][i] for i in top_items]

#### 2-5. 추천 결과 보기

In [40]:
user_id = 3
recommended = recommend_hybrid(model_hybrid, user_id)

print(f" Hybrid 모델 - 사용자 {user_id}에게 추천하는 영화:")
for i, movie in enumerate(recommended, 1):
    print(f"{i}. {movie}")

 Hybrid 모델 - 사용자 3에게 추천하는 영화:
1. Empire Strikes Back, The (1980)
2. From Dusk Till Dawn (1996)
3. Army of Darkness (1993)
4. Muppet Treasure Island (1996)
5. Diva (1981)
